In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch # Importing pytorch
from torchvision import models, transforms # To get pretrained deep NNs and various image transforms
from torch.utils.data import Dataset, DataLoader #
from torch import nn, optim # NN module to customize network and optim to get various optimizers
from PIL import Image # to load images
import matplotlib.pyplot as plt # to plot images and graphs

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Custom dataset class
class DogBreedDataset(Dataset):
    def __init__(self, training_dir_path, train_df, label2id, transform):
        self.training_dir_path = training_dir_path
        self.df = train_df
        
        # Calculate weights for each class to pass to Cross Entropy Loss
        temp = train_df.replace({"breed": label2id})
        cnts_df = temp.groupby(['breed']).count()
        self.weights = cnts_df.id.values.tolist()
        total = sum(self.weights)
        self.weights = list(map(lambda w: (total/w)/100, self.weights))
        
        self.label2id = label2id
        self.transform = transform
        
    def __len__(self):
        # Return size of the dataset
        return len(self.df.index)
    
    def __getitem__(self, idx):
        # Read image using PIL
        image = Image.open(os.path.join(self.training_dir_path, self.df.iloc[idx, 0]) + '.jpg')
        
        # Extract image's true label from dataframe
        label = label2id[self.df.iloc[idx, 1]]
        
        # Transform image if specified (eg. Resize, Crop, flip, etc)
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
# Get resnet with pre-trained weights and modify classifier to output N numbers
model = models.resnext50_32x4d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 120)
i=0
for name, param in model.named_parameters():
    # print(i, name)
    if i < 93:
        param.requires_grad = False
    else:
        break
    i += 1

device = "cuda" if torch.cuda.is_available() else "cpu"

# Move model to device
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth


In [4]:
# Read csv file containing image file names and it's corresponding breed name. Also shuffle after reading
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv').sample(frac=1).reset_index(drop=True)

# Get all unique breed names
unique_labels = labels.breed.unique()
unique_labels.sort()

# Make dict to map breed name to a index b.w 0 to N-1
label2id = {}
for idx, name in enumerate(unique_labels):
    label2id[name] = idx

training_dir_path = '/kaggle/input/dog-breed-identification/train'
labels_path = '/kaggle/input/dog-breed-identification/labels.csv'
    
def get_train_val_loader(labels, label2id, training_dir_path, labels_path, train_frac = 1, batch_size = 32, nb_workers = 4):
    # Total number of training examples
    nb_exmpls = len(labels.index)
    
    # Split original dataframe to train and validation df
    split = int(np.floor(train_frac * nb_exmpls))
    
    # Shuffle data
    labels = labels.sample(frac = 1)
    
    # Train/Val split
    train_df = labels.iloc[:split, :]
    val_df = labels.iloc[split:, :]

    # Transform for train set
    train_transform = transforms.Compose([transforms.ColorJitter(brightness=0.3, contrast=0.4, saturation=0.4),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomAffine(degrees=30, translate=(.2, .2), scale=(0.75, 1.25),
                                                      shear=[-30, 30, -30, 30]),
                                        transforms.Resize((224, 224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    # Create dataset class for train set
    train_dataset = DogBreedDataset(training_dir_path, train_df, label2id, train_transform)
    
    # Create train data loader to batch data on the fly to feed the model
    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=nb_workers, shuffle=True)
    
    if train_frac == 1:
        return (train_dataset, train_loader)
    else:
        # Transform for validation set
        val_transform = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

        # Create dataset class for validation set
        val_dataset = DogBreedDataset(training_dir_path, val_df, label2id, val_transform)

        # Data loader for val set
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=nb_workers, shuffle=True)
        
        return (train_dataset, train_loader, val_dataset, val_loader)
    
batch_size = 256
train_dataset, train_loader, val_dataset, val_loader = get_train_val_loader(labels, label2id, training_dir_path, labels_path, batch_size = batch_size, train_frac = 0.8)

In [5]:
# for images, targets in train_loader:
#     for i in range(batch_size):
#         img = images[i, :, :, :].numpy().transpose((1, 2, 0))
#         mean = np.array([[[.485, 0.456, 0.406]]])
#         std = np.array([[[.229, 0.224, 0.225]]])
#         img = img*std + mean
#         plt.imshow(img)
#         plt.title(unique_labels[int(targets[i].item())])
#         plt.show()
#     break

In [6]:
# Cross entropy loss for multiclass classification and Adam optimizer
criterion = nn.CrossEntropyLoss(weight = torch.cuda.FloatTensor(train_dataset.weights))
optimizer = optim.Adam(filter(lambda param: param.requires_grad, model.parameters()))

# Learning rate scheduler to update lr when loss stops improving
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

In [7]:
# Total number of epochs to train for
n_epochs = 100

# Keep track of max val_acc
val_acc_max = 0

for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    
    # Set model to train mode
    model.train()
    
    # Iterate through training set by processing batches
    for data, target in train_loader:
        # Move data to device
        data, target = data.to(device), target.to(device)
        
        # Zero out previous gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Calculate loss
        loss = criterion(output, target)
        
        # Backpropagate to calculate gradients w.r.t. loss
        loss.backward()
        
        # Alter model params
        optimizer.step()
        
        train_loss += loss.item()*data.size(0)
        
    # Calculate avg loss over whole training set
    train_loss = train_loss/len(train_dataset)
    
    # Set model to eval mode
    model.eval()
    
    # Turn off gradient calculations for validation task
    with torch.no_grad():
        
        # For calculating val acc
        total = 0
        correct = 0

        # Do forward pass through validation data and calculate val_loss
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, preds = torch.max(output, dim=1)
            total += target.size(0)
            correct += torch.sum(preds == target).cpu().data.item()
            loss = criterion(output, target)
            val_loss += loss.item()*data.size(0)

        val_loss = val_loss/len(val_dataset)
        
    print('Epoch: {} \Train Loss: {:.6f} \t Val Loss: {:.6f} \t Val Acc: {:.6f}'.format(epoch+1, train_loss, val_loss, correct/total))
    
    # Save model if val_acc has increased
    if (correct/total) > val_acc_max:
        print('Val Acc increased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_acc_max, correct/total))
        torch.save(model, 'dog_breed_model.pt')
        val_acc_max = correct/total
        
    scheduler.step(val_loss)

Epoch: 1 \Train Loss: 2.793475 	 Val Loss: 2.107004 	 Val Acc: 0.461125
Val Acc increased (0.000000 --> 0.461125).  Saving model ...
Epoch: 2 \Train Loss: 1.760453 	 Val Loss: 1.865718 	 Val Acc: 0.502200
Val Acc increased (0.461125 --> 0.502200).  Saving model ...
Epoch: 3 \Train Loss: 1.470776 	 Val Loss: 1.547243 	 Val Acc: 0.552567
Val Acc increased (0.502200 --> 0.552567).  Saving model ...
Epoch: 4 \Train Loss: 1.259667 	 Val Loss: 1.462246 	 Val Acc: 0.593154
Val Acc increased (0.552567 --> 0.593154).  Saving model ...
Epoch: 5 \Train Loss: 1.124360 	 Val Loss: 1.444758 	 Val Acc: 0.600489
Val Acc increased (0.593154 --> 0.600489).  Saving model ...
Epoch: 6 \Train Loss: 1.024087 	 Val Loss: 1.393339 	 Val Acc: 0.610758
Val Acc increased (0.600489 --> 0.610758).  Saving model ...
Epoch: 7 \Train Loss: 0.921128 	 Val Loss: 1.405163 	 Val Acc: 0.617604
Val Acc increased (0.610758 --> 0.617604).  Saving model ...
Epoch: 8 \Train Loss: 0.816486 	 Val Loss: 1.430312 	 Val Acc: 0.6430

KeyboardInterrupt: 

In [10]:
test_dir_path = '/kaggle/input/dog-breed-identification/test'
filenames = []
for filename in os.listdir(test_dir_path):
    filenames.append(filename.split('.')[0])
out = pd.DataFrame(filenames, columns = ['id'])
out = out.sort_values(by = 'id')
out = out.reset_index(drop=True)

In [11]:
# Custom dataset class
class TestDataset(Dataset):
    def __init__(self, test_dir_path, test_df, transform):
        self.test_dir_path = test_dir_path
        self.df = test_df
        self.transform = transform
        
    def __len__(self):
        # Return size of the dataset
        return len(self.df.index)
    
    def __getitem__(self, idx):
        # Read image using PIL
        image = Image.open(os.path.join(self.test_dir_path, self.df.iloc[idx, 0]) + '.jpg')
        
        # Transform image if specified (eg. Resize, Crop, flip, etc)
        if self.transform:
            image = self.transform(image)
        return image
    
test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
batch_size = 128
test_dataset = TestDataset(test_dir_path, out, test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=0)

In [12]:
model_path = '/kaggle/working/dog_breed_model.pt'

preds = np.zeros((batch_size, len(unique_labels)), dtype=np.float32)

saved_model = torch.load(model_path)
saved_model.to(device)
saved_model.eval()

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        output = saved_model(data)
        output = torch.nn.functional.softmax(output, 1)
        preds = np.vstack((preds, output.detach().cpu().numpy()))
    preds = preds[batch_size:, :]

In [16]:
preds_df = pd.DataFrame(data=preds, columns=unique_labels)

In [35]:
submission = pd.concat([out, preds_df], axis = 1)
submission.to_csv (r'/kaggle/working/submission.csv', index = False, header=True)